In [1]:
include("../code/julia/ha-trade.jl")
include("../code/julia/ha-trade-welfare.jl")

using MINPACK
using Plots
using CSV
using DataFrames

### First step, match up on no quality version...

In [2]:
γ = 1.5 # curvatuve on CRRA utility function
σϵ = 0.25 # logit dispersion parameter
Ncntry = 2 # number of countries

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ, ψslope = 0.0);

#ψslope controls how residual demand varies with income state...

In [3]:
# here are some simpe country parametrers
TFP = [1.0; 1.0]

τ = [0.0; 0.0]

L = [1.0; 1.0]

d_ij = 1.745

d = [1.0 d_ij; d_ij 1.0]

# this sets up the country specific paramters
cntry_prm = country_params(Ncntry = Ncntry, L = L, d = d, TFP = TFP);

In [4]:
f(x) = world_equillibrium_FG(exp.(x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the finacial globalization case

function f!(fvec, x)

    fvec .= f(x)

end


xguess = [1.0; 1.02]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, log.(xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

# This solver works very well in general. Spencer Lyon converted old-school minpack
# routines in C into julia

print(sol)

wage = [exp.(sol.x[1]); 1.0]
R = exp.(sol.x[2])

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.560862e-01     0.000000e+00        39.267000
     2     1.456059e-02     2.654986e-05         1.122000
     3     1.638137e-03     4.506004e-07         0.136000
     4     1.812301e-05     7.164351e-09         0.149000
     5     1.674584e-08     9.008308e-13         0.135000
     6     6.698718e-11     8.321748e-19         0.150000
     7     3.676642e-13     1.185695e-23         0.144000
     8     1.179612e-15     3.365318e-28         0.137000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [0.0, 0.01980262729617973]
 * Zero: [-2.039586404137559e-9, 0.014094023815588037]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 41.240000 seconds
 * Function Calls: 8
 * Jacobian Calls (df/dx): 1

1.0141938128276817

In [5]:
Rsol = [R; R]

Y, tradeflows, A_demand, Gbudget, tradeshare, hh, dist = world_equillibrium(Rsol, wage, τ, hh_prm, cntry_prm, tol_vfi = 1e-10);

In [6]:
R

1.0141938128276817

In [7]:
tradeshare

2×2 Matrix{Float64}:
 0.900484   0.0995163
 0.0995163  0.900484

In [8]:
hh[1].πprob[:,:,2]
# guys in country one, a X z, probability of buying from 2

100×10 Matrix{Float64}:
 0.00747609  0.0192432  0.0202433  0.0386541  …  0.103814  0.123432  0.138619
 0.0139284   0.0212163  0.0281047  0.0428235     0.105277  0.127093  0.139288
 0.0194756   0.0264393  0.0357197  0.0456883     0.106613  0.129608  0.139941
 0.0218411   0.0282292  0.0387705  0.0478642     0.107806  0.131053  0.140583
 0.0265513   0.0302667  0.042976   0.0506709     0.108911  0.132266  0.141216
 0.0285041   0.0333028  0.0457674  0.0530594  …  0.109967  0.133278  0.141837
 0.030625    0.0352261  0.0480061  0.0548275     0.111015  0.134253  0.142449
 0.0333905   0.0371223  0.0507749  0.0564595     0.112049  0.135159  0.143062
 0.0354688   0.039016   0.0531147  0.0582905     0.112984  0.136008  0.143643
 0.037335    0.0409391  0.0548801  0.0601975     0.113907  0.136789  0.144216
 0.0392264   0.0426161  0.0565561  0.061869   …  0.114807  0.137517  0.14478
 0.0411777   0.0442844  0.058384   0.0633612     0.115682  0.138219  0.145336
 0.0428169   0.0459213  0.0602438  0.0647

### Now turn quality on...

In [9]:

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ, ψslope = 0.2);

#ψslope controls how residual demand varies with income state...

In [10]:
f(x) = world_equillibrium_FG(exp.(x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the finacial globalization case

function f!(fvec, x)

    fvec .= f(x)

end


xguess = [1.0; 1.02]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, log.(xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

# This solver works very well in general. Spencer Lyon converted old-school minpack
# routines in C into julia

print(sol)

wage = [exp.(sol.x[1]); 1.0]
R = exp.(sol.x[2])

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.992083e-01     0.000000e+00         0.114000
     2     2.253156e-02     4.054422e-05         0.353000
     3     3.170333e-03     1.074674e-06         0.087000
     4     4.916331e-05     2.946856e-08         0.117000
     5     1.685977e-07     7.182151e-12         0.117000
     6     1.002796e-10     8.588696e-17         0.120000
     7     4.424239e-13     2.833389e-23         0.152000
     8     3.289036e-15     5.500317e-28         0.129000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [0.0, 0.01980262729617973]
 * Zero: [-2.2768317339848573e-9, 0.01258038097184609]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 1.189000 seconds
 * Function Calls: 8
 * Jacobian Calls (df/dx): 1

1.012659846851913

In [11]:
Rsol = [R; R]

Y, tradeflows, A_demand, Gbudget, tradeshare, hh, dist = world_equillibrium(Rsol, wage, τ, hh_prm, cntry_prm, tol_vfi = 1e-10);

In [12]:
R

1.012659846851913

In [13]:
tradeshare

2×2 Matrix{Float64}:
 0.912448   0.0875522
 0.0875522  0.912448

In [14]:
hh[1].πprob[:,:,2]
# guys in country one, a X z, probability of buying from 2

100×10 Matrix{Float64}:
 0.0203927  0.0360656  0.0380981  …  0.069461   0.0833217  0.0675049
 0.0374807  0.0397536  0.0524594     0.0704901  0.0860155  0.0678605
 0.0517403  0.0489637  0.0661095     0.071409   0.0879135  0.06821
 0.057837   0.0523315  0.0715266     0.0722387  0.089045   0.0685533
 0.0693525  0.0560704  0.0789035     0.0730149  0.0898944  0.0688905
 0.0744127  0.0612837  0.08379    …  0.0737619  0.0906548  0.0692287
 0.0797019  0.0647962  0.087817      0.0745066  0.0913656  0.0695538
 0.0862534  0.0682007  0.0925053     0.0752162  0.0920282  0.0698665
 0.0913775  0.0715594  0.096533      0.0758752  0.0926106  0.0701749
 0.0959336  0.0749256  0.0996836     0.0765248  0.0931592  0.0704782
 0.100463   0.0778429  0.10262    …  0.0771596  0.0936835  0.0707768
 0.105026   0.0807764  0.105696      0.077774   0.094189   0.0710721
 0.108972   0.0836368  0.108843      0.0783727  0.0946961  0.0713645
 ⋮                                ⋱                        
 0.25642    0.194793 